In [1]:
import numpy as np
import os
import cv2
import pickle
import tensorflow as tf
import time
from tensorflow.python.keras.backend import learning_phase
from sklearn.model_selection import train_test_split



In [2]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(64,64,3)),
        tf.keras.layers.Conv2D(32,(3,3),activation='relu'), #filters -> how many weighted values are in the filter. 32 filters of 3x3 size.
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.MaxPool2D(), # takes the out. of conv. and keeps only the max of the windows size
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.GlobalAvgPool2D(), # computes avg according to some axis
        tf.keras.layers.Dense(64,activation='relu'), # matrix-vector multiplication. Changes dimesions of vector.
        tf.keras.layers.Dense(8,activation='softmax') # gives us 10 probabilities that sum up to 1. We have 10 numbers. -> Output

    ]
)

def classification(x_train,y_train,model,path="output"):
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=5, validation_split=0.1)
    model.save(path+'/model')

def loadData(path="output"):
    '''
    Loads the data to be used by the classifiers
    '''
    pickle_in = open(path+"/X.pickle", "rb")
    X = pickle.load(pickle_in)
    pickle_in = open(path+"/y.pickle", "rb")
    y = pickle.load(pickle_in)
    pickle_in = open(path+"/data.pickle", "rb")
    data = pickle.load(pickle_in)

    return X,y,data

In [3]:
X,y,_ = loadData()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('Training Size:', len(X_train))
print('Testing  Size:', len(X_test))

Training Size: 8363
Testing  Size: 2091


In [4]:
classification(np.asarray(X_train),np.asarray(y_train),model)

Epoch 1/5
236/236 [==============================] - 65s 274ms/step - loss: 0.9598 - accuracy: 0.6199 - val_loss: 1.0357 - val_accuracy: 0.5615
Epoch 2/5
236/236 [==============================] - 59s 250ms/step - loss: 0.6238 - accuracy: 0.7397 - val_loss: 0.7431 - val_accuracy: 0.6655
Epoch 3/5
236/236 [==============================] - 62s 262ms/step - loss: 0.5013 - accuracy: 0.7966 - val_loss: 0.5711 - val_accuracy: 0.7240
Epoch 4/5
236/236 [==============================] - 59s 249ms/step - loss: 0.4179 - accuracy: 0.8366 - val_loss: 1.0164 - val_accuracy: 0.6153
Epoch 5/5
236/236 [==============================] - 59s 249ms/step - loss: 0.3536 - accuracy: 0.8657 - val_loss: 0.4238 - val_accuracy: 0.8471
INFO:tensorflow:Assets written to: output/model\assets


In [5]:
results = model.evaluate(np.asarray(X_test),np.asarray(y_test))
print(results)

66/66 [==============================] - 5s 73ms/step - loss: 0.4392 - accuracy: 0.8436
[0.4391638934612274, 0.8436154723167419]
